##### Import statements

In [1]:
import numpy as np
from tqdm import tqdm

##### The problem

> Train Set

In [2]:
train_X = [
	"good",
	"bad",
	"happy",
	"sad",
	"not good",
	"not bad",
	"not happy",
	"not sad",
	"very good",
	"very bad",
	"very happy",
	"very sad",
	"i am happy",
	"this is good",
	"i am bad",
	"this is bad",
	"i am sad",
	"this is sad",
	"i am not happy",
	"this is not good",
	"i am not bad",
	"this is not sad",
	"i am very happy",
	"this is very good",
	"i am very bad",
	"this is very sad",
	"this is very happy",
	"i am good not bad",
	"this is good not bad",
	"i am bad not good",
	"i am good and happy",
	"this is not good and not happy",
	"i am not at all good",
	"i am not at all bad",
	"i am not at all happy",
	"this is not at all sad",
	"this is not at all happy",
	"i am good right now",
	"i am bad right now",
	"this is bad right now",
	"i am sad right now",
	"i was good earlier",
	"i was happy earlier",
	"i was bad earlier",
	"i was sad earlier",
	"i am very bad right now",
	"this is very good right now",
	"this is very sad right now",
	"this was bad earlier",
	"this was very good earlier",
	"this was very bad earlier",
	"this was very happy earlier",
	"this was very sad earlier",
	"i was good and not bad earlier",
	"i was not good and not happy earlier",
	"i am not at all bad or sad right now",
	"i am not at all good or happy right now",
	"this was not happy and not good earlier"
]

train_y = [
	1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0
]

> Test Set

In [3]:
test_X = [
	"this is happy",
	"i am good",
	"this is not happy",
	"i am not good",
	"this is not bad",
	"i am not sad",
	"i am very good",
	"this is very bad",
	"i am very sad",
	"this is bad not good",
	"this is good and happy",
	"i am not good and not happy",
	"i am not at all sad",
	"this is not at all good",
	"this is not at all bad",
	"this is good right now",
	"this is sad right now",
	"this is very bad right now",
	"this was good earlier",
	"i was not happy and not good earlier"
]

test_y = [1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0]

In [4]:
vocab = set( [ q for text in train_X for q in text.split()])
vocab_size = len( vocab)

word_to_index = { w: i for i, w in enumerate( vocab)}

In [5]:
word_to_index

{'happy': 0,
 'and': 1,
 'is': 2,
 'right': 3,
 'or': 4,
 'sad': 5,
 'i': 6,
 'bad': 7,
 'good': 8,
 'at': 9,
 'all': 10,
 'not': 11,
 'earlier': 12,
 'now': 13,
 'was': 14,
 'am': 15,
 'very': 16,
 'this': 17}

##### Utility functions

In [6]:
# Helper Function
def oneHotEncode( text):
	inputs = []

	for q in text.split():
		vector = np.zeros( ( 1, vocab_size))
		vector[ 0][ word_to_index[ q]] = 1
		inputs += [ vector]

	return inputs

# Xavier Normalized Initialization
def initWeight( input_size, output_size):
	return np.random.uniform( -1, 1, ( input_size, output_size)) * np.sqrt( 6 / ( input_size + output_size))

##### Activation functions

In [7]:
def tanh( input, derivative = False):
	if derivative:
		# return 1 - ( tanh( input) ** 2)
		# because we will store the value of tanh(x) in network, instead of just x
		return 1 - ( input ** 2)
	
	return np.tanh( input)

def softmax( input):
	return np.exp( input) / np.sum( np.exp( input))

##### Recurrent Neural Network Class

In [8]:
class RNN:
	def __init__( self, input_size, hidden_size, output_size):
		# Network weight
  
		# input weights
		self.w1 = initWeight( input_size, hidden_size)
		# hidden state weights
		self.w2 = initWeight( hidden_size, hidden_size)
		# output weights
		self.w3 = initWeight( hidden_size, output_size)

		# biases

		# there is no bias for first layer ( b2 would suffice for hidden state function)

		# hidden state bias
		self.b2 = np.zeros( ( 1, hidden_size))
		# output bias
		self.b3 = np.zeros( ( 1, output_size))

	# Forward Propagation
	def forward( self, inputs):

		self.hidden_states = [ np.zeros_like( self.b2)]

		for input in inputs:
			layer1_output = np.dot( input, self.w1)
			layer2_output = np.dot( self.hidden_states[ -1], self.w2) + self.b2

			self.hidden_states += [ tanh( layer1_output + layer2_output)]

		return np.dot( self.hidden_states[ -1], self.w3) + self.b3
	
	# Backward Propagation
	def backward( self, error, input, learning_rate):
		d_b3 = error
		d_w3 = np.dot( self.hidden_states[ -1].T, error)

		d_b2 = np.zeros_like( self.b2)
		d_w2 = np.zeros_like( self.w2)
		d_w1 = np.zeros_like( self.w1)

		d_hidden_state = np.dot( error, self.w3.T)

		for q in reversed( range( len( input))):
			d_hidden_state *= tanh( self.hidden_states[ q + 1], derivative=True)

			d_b2 += d_hidden_state

			d_w2 += np.dot( self.hidden_states[ q].T, d_hidden_state)

			d_w1 += np.dot( input[ q].T, d_hidden_state)

			d_hidden_state = np.dot( d_hidden_state, self.w2)

		for d_ in ( d_b3, d_w3, d_b2, d_w2, d_w1):
			np.clip( d_, -1, 1, out=d_)

		self.b3 += learning_rate * d_b3
		self.w3 += learning_rate * d_w3
		self.b2 += learning_rate * d_b2
		self.w2 += learning_rate * d_w2
		self.w1 += learning_rate * d_w1

	# Training Method
	def fit( self, inputs, labels, epochs, learning_rate):
		for _ in tqdm( range( epochs)):
			for input, label in zip( inputs, labels):
				input = oneHotEncode( input)

				prediction = self.forward( input)

				error = -softmax( prediction)

				error[ 0][ label] += 1

				self.backward( error, input, learning_rate)

	def predict( self, inputs, labels):
		hit = 0

		for input, label in zip( inputs, labels):
			print( input)

			input = oneHotEncode( input)
			prediction = self.forward( input)

			print( [ "Negative", "Positive"][ np.argmax( prediction)], end="\n\n")

			if np.argmax( prediction) == label:
				hit += 1

		print( f"Accuracy: { hit / len( inputs) * 100}%")



In [9]:
rnn = RNN(
	input_size=vocab_size,
	hidden_size=64,
	output_size=2
)

rnn.fit(
	inputs=train_X,
	labels=train_y,
	epochs=1000,
	learning_rate=0.02
)

rnn.predict(
	inputs=test_X,
	labels=test_y
)

100%|██████████| 1000/1000 [00:14<00:00, 71.18it/s]

this is happy
Positive

i am good
Positive

this is not happy
Negative

i am not good
Negative

this is not bad
Positive

i am not sad
Positive

i am very good
Positive

this is very bad
Negative

i am very sad
Negative

this is bad not good
Negative

this is good and happy
Positive

i am not good and not happy
Negative

i am not at all sad
Positive

this is not at all good
Negative

this is not at all bad
Positive

this is good right now
Positive

this is sad right now
Negative

this is very bad right now
Negative

this was good earlier
Positive

i was not happy and not good earlier
Negative

Accuracy: 100.0%
